In [1]:
import torch
import os
from torch.utils.data import DataLoader
from dataset.dataset import get_cdiscount_dataset
from model.model import assemble_model, assemble_model_with_classifier
from predictor.predictor import get_predictor

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
config = {
    'test_batch_size': 128,
    'arch': 'vgg19_bn',
    'checkpoint': 'vgg19_bn_checkpoint_epoch_0_iter_63000.pth.tar',
    'print_freq': 10,
    'pred_filename': "vgg19_bn-Epoch_0-iter-63000.csv"
}

In [3]:
import torchvision.models as models

# get dataset
print('getting dataset...')
test_dataset = get_cdiscount_dataset(offsets_csv="test_offsets.csv",
                                     images_csv="test_images.csv",
                                     bson_file_path="/mnt/data/cdiscount/test.bson",
                                     with_label=False,
                                     resize=160)

# get data loader
print('getting data loader...')
test_dataloader = DataLoader(test_dataset, batch_size=config['test_batch_size'], shuffle=False, num_workers=6)

# define model
print("=> using pre-trained model '{}'".format(config['arch']))
model = models.__dict__[config['arch']](pretrained=False)

classifier_layer = [
    torch.nn.Linear(in_features=12800, out_features=5270)
]

classifier = torch.nn.Sequential(*classifier_layer)

model = assemble_model_with_classifier(model, -1, classifier)
model = torch.nn.DataParallel(model).cuda()
print(model)


# load checkpoint
if not os.path.isfile(config['checkpoint']):
    print("=> no checkpoint found at '{}'".format(config['checkpoint']))
    
print("=> loading checkpoint '{}'".format(config['checkpoint']))
checkpoint = torch.load(config['checkpoint'])
model.load_state_dict(checkpoint['state_dict'])
print("=> loaded checkpoint:")

print('Epoch: [{0}][{1}]\t'
      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
      'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
       checkpoint['epoch'], checkpoint['iter'], loss=checkpoint['loss'], top1=checkpoint['top1'], top5=checkpoint['top5']))

del checkpoint # save some GPU memory

# get trainer
Predictor = get_predictor(test_dataloader, model, config)

# Run!
Predictor.run()

getting dataset...
getting data loader...
=> using pre-trained model 'vgg19_bn'
DataParallel (
  (module): AssembledModel (
    (model): Sequential (
      (0): Sequential (
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (2): ReLU (inplace)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (5): ReLU (inplace)
        (6): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
        (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
        (9): ReLU (inplace)
        (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
        (12): ReLU (inplace)
        (13): MaxPool2d (size=(2, 2), stride=(2, 2), d

RuntimeError: cuda runtime error (2) : out of memory at /pytorch/torch/lib/THC/generic/THCStorage.cu:66